# ⚽ Football Analysis Pipeline

This notebook runs the football analysis CV pipeline with GPU acceleration.

**Features:**
- YOLOv8 player/ball/goalkeeper detection
- ByteTrack object tracking
- SigLIP + UMAP + KMeans team classification
- Pitch keypoint detection and homography
- Analytics computation and event detection

**Requirements:**
- Google Colab with GPU runtime (T4 or better)
- ~10GB disk space for models

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q ultralytics supervision opencv-python-headless
!pip install -q transformers umap-learn scikit-learn
!pip install -q fastapi uvicorn python-multipart httpx
!pip install -q tqdm pillow python-dotenv

In [ ]:
# Clone the pipeline code
!git clone https://github.com/YOUR_USERNAME/football-dashboard.git
%cd football-dashboard/pipeline

In [ ]:
# Download YOLOv8x model
!mkdir -p models
!wget -q -O models/yolov8x.pt https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8x.pt
print("Model downloaded!")

In [ ]:
# Upload your custom models (optional)
from google.colab import files

print("Upload your custom ball_detection.pt and pitch_detection.pt models:")
uploaded = files.upload()

for filename in uploaded.keys():
    !mv "{filename}" models/
    print(f"Moved {filename} to models/")

In [ ]:
# Upload a video for analysis
from google.colab import files

print("Upload your football match video:")
uploaded = files.upload()

video_path = list(uploaded.keys())[0]
print(f"Video uploaded: {video_path}")

In [ ]:
# Initialize the processor
from processor import FootballProcessor, ProcessingConfig

config = ProcessingConfig(
    mode="all",
    device="cuda",
    det_model_path="models/yolov8x.pt",
    ball_model_path="models/ball_detection.pt" if os.path.exists("models/ball_detection.pt") else None,
    pitch_model_path="models/pitch_detection.pt" if os.path.exists("models/pitch_detection.pt") else None,
)

processor = FootballProcessor(config)
print("Processor initialized!")

In [ ]:
# Load video
frames = processor.load_video(video_path)
print(f"Loaded {len(frames)} frames")

In [ ]:
# Run detection
detections = processor.detect(frames)
print(f"Detection complete!")

In [ ]:
# Run tracking
tracks = processor.track(frames, detections)
print(f"Tracking complete!")
print(f"Players tracked: {len(set(pid for frame in tracks['players'] for pid in frame.keys()))}")

In [ ]:
# Run team classification
tracks = processor.classify_teams(frames, tracks)
print(f"Team classification complete!")
print(f"Team colors: {processor.team_colors}")

In [ ]:
# Run pitch detection
pitch_data = processor.detect_pitch(frames)
print(f"Pitch detection complete!")

In [ ]:
# Compute homography
transformers = processor.compute_homography(pitch_data)
valid_count = sum(1 for t in transformers if t is not None)
print(f"Homography computed for {valid_count}/{len(transformers)} frames")

In [ ]:
# Compute analytics
analytics = processor.compute_analytics(tracks, transformers)
print("\n=== Analytics Summary ===")
print(f"Team 1 Possession: {analytics['possession']['team_1_percentage']:.1f}%")
print(f"Team 2 Possession: {analytics['possession']['team_2_percentage']:.1f}%")
print(f"Ball detection rate: {analytics['ball_stats']['detection_rate']*100:.1f}%")

In [ ]:
# Detect events
events = processor.detect_events(tracks, analytics)
print(f"\nDetected {len(events)} events:")
for event in events[:10]:
    print(f"  - {event['type']} at {event['time']:.1f}s")

In [ ]:
# Render output videos
output_urls = processor.render_outputs(
    frames, tracks, pitch_data, transformers, "colab_analysis"
)
print(f"\nOutput videos:")
print(f"  Annotated: {output_urls['annotated']}")
print(f"  Radar: {output_urls['radar']}")

In [ ]:
# Display annotated video
from IPython.display import HTML
from base64 import b64encode

video_path = output_urls['annotated']
video_data = open(video_path, 'rb').read()
video_b64 = b64encode(video_data).decode()

HTML(f'''
<video width="800" controls>
    <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
</video>
''')

In [ ]:
# Download results
import json

# Save tracking data
with open('/tmp/tracks.json', 'w') as f:
    # Convert numpy arrays to lists for JSON serialization
    json.dump(tracks, f, default=lambda x: x.tolist() if hasattr(x, 'tolist') else str(x))

# Save analytics
with open('/tmp/analytics.json', 'w') as f:
    json.dump(analytics, f, default=lambda x: x.tolist() if hasattr(x, 'tolist') else str(x))

# Download files
from google.colab import files
files.download(output_urls['annotated'])
files.download('/tmp/tracks.json')
files.download('/tmp/analytics.json')

## 🚀 Start Pipeline API Server

Run the cell below to start the FastAPI server for integration with the dashboard.

In [ ]:
# Start the API server (use ngrok for public access)
!pip install -q pyngrok

from pyngrok import ngrok
import threading
import uvicorn

# Start server in background
def run_server():
    from main import app
    uvicorn.run(app, host="0.0.0.0", port=8001)

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Create public URL
public_url = ngrok.connect(8001)
print(f"\n🌐 Pipeline API available at: {public_url}")
print(f"\nUse this URL as the callback URL in your dashboard settings.")